In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

#smooth = 1e-12

# def jaccard_coef(y_true, y_pred):
#     intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

#     jac = (intersection + smooth) / (sum_ - intersection + smooth)

#     return K.mean(jac)


# def jaccard_coef_int(y_true, y_pred):
#     y_pred_pos = K.round(K.clip(y_pred, 0, 1))

#     intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

#     jac = (intersection + smooth) / (sum_ - intersection + smooth)

#     return K.mean(jac)


# def jaccard_coef_loss(y_true, y_pred):
#     return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
# #    return -K.log(jaccard_coef(y_true, y_pred))

smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = BatchNormalization(axis=1)(c6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
    c6 = BatchNormalization(axis=1)(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/losstest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=dice_coef_loss,\
                      metrics=['binary_crossentropy', dice_coef, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, validation_data=(array([[[..., epochs=50, steps_per_epoch=100, callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 25s 251ms/step - loss: -0.4037 - binary_crossentropy: 0.8262 - dice_coef: 0.4037 - binary_accuracy: 0.7023 - val_loss: -0.0601 - val_binary_crossentropy: 0.8672 - val_dice_coef: 0.0601 - val_binary_accuracy: 0.7859
Epoch 2/50
100/100 [==============================] - 17s 166ms/step - loss: -0.4694 - binary_crossentropy: 0.9023 - dice_coef: 0.4694 - binary_accuracy: 0.7612 - val_loss: -0.1959 - val_binary_crossentropy: 1.3847 - val_dice_coef: 0.1959 - val_binary_accuracy: 0.7559
Epoch 3/50
100/100 [==============================] - 17s 166ms/step - loss: -0.4839 - binary_crossentropy: 0.9595 - dice_coef: 0.4839 - binary_accuracy: 0.7879 - val_loss: -0.0539 - val_binary_crossentropy: 1.3287 - val_dice_coef: 0.0539 - val_binary_accuracy: 0.7689
Epoch 4/50
100/100 [==============================] - 16s 164ms/step - loss: -0.5161 - binary_crossentropy: 0.9703 - dice_coef: 0.5161 - binary_accuracy: 0.8043 - val_loss: -0.4579 - val_binary

Epoch 9/50
100/100 [==============================] - 19s 188ms/step - loss: -0.2405 - binary_crossentropy: 0.1017 - dice_coef: 0.2405 - binary_accuracy: 0.9907 - val_loss: -0.1572 - val_binary_crossentropy: 0.1815 - val_dice_coef: 0.1572 - val_binary_accuracy: 0.9834
Epoch 10/50
100/100 [==============================] - 19s 189ms/step - loss: -0.2479 - binary_crossentropy: 0.1058 - dice_coef: 0.2479 - binary_accuracy: 0.9905 - val_loss: -0.1616 - val_binary_crossentropy: 0.1594 - val_dice_coef: 0.1616 - val_binary_accuracy: 0.9851
Epoch 11/50
100/100 [==============================] - 19s 189ms/step - loss: -0.2635 - binary_crossentropy: 0.1089 - dice_coef: 0.2635 - binary_accuracy: 0.9903 - val_loss: -0.1743 - val_binary_crossentropy: 0.2096 - val_dice_coef: 0.1743 - val_binary_accuracy: 0.9823
Epoch 12/50
100/100 [==============================] - 19s 190ms/step - loss: -0.2435 - binary_crossentropy: 0.1128 - dice_coef: 0.2435 - binary_accuracy: 0.9902 - val_loss: -0.0687 - val_bin

Epoch 17/50
100/100 [==============================] - 19s 188ms/step - loss: -0.1147 - binary_crossentropy: 0.2925 - dice_coef: 0.1147 - binary_accuracy: 0.9738 - val_loss: -0.0706 - val_binary_crossentropy: 0.4443 - val_dice_coef: 0.0706 - val_binary_accuracy: 0.9672
Epoch 18/50
100/100 [==============================] - 19s 189ms/step - loss: -0.1319 - binary_crossentropy: 0.2783 - dice_coef: 0.1319 - binary_accuracy: 0.9762 - val_loss: -3.4324e-04 - val_binary_crossentropy: 0.0855 - val_dice_coef: 3.4324e-04 - val_binary_accuracy: 0.9919
Epoch 19/50
100/100 [==============================] - 19s 187ms/step - loss: -0.1382 - binary_crossentropy: 0.2615 - dice_coef: 0.1382 - binary_accuracy: 0.9771 - val_loss: -0.0465 - val_binary_crossentropy: 0.1521 - val_dice_coef: 0.0465 - val_binary_accuracy: 0.9846
Epoch 20/50
100/100 [==============================] - 19s 189ms/step - loss: -0.1353 - binary_crossentropy: 0.2961 - dice_coef: 0.1353 - binary_accuracy: 0.9758 - val_loss: -0.1097 

...

Epoch 19/50
100/100 [==============================] - 17s 172ms/step - loss: -0.3351 - binary_crossentropy: 1.3686 - dice_coef: 0.3351 - binary_accuracy: 0.8757 - val_loss: -0.1557 - val_binary_crossentropy: 1.3508 - val_dice_coef: 0.1557 - val_binary_accuracy: 0.8754
Epoch 20/50
100/100 [==============================] - 17s 168ms/step - loss: -0.3379 - binary_crossentropy: 1.3735 - dice_coef: 0.3379 - binary_accuracy: 0.8788 - val_loss: -0.1106 - val_binary_crossentropy: 0.9664 - val_dice_coef: 0.1106 - val_binary_accuracy: 0.9088
Epoch 21/50
100/100 [==============================] - 17s 169ms/step - loss: -0.3384 - binary_crossentropy: 1.3470 - dice_coef: 0.3384 - binary_accuracy: 0.8833 - val_loss: -0.1799 - val_binary_crossentropy: 1.3934 - val_dice_coef: 0.1799 - val_binary_accuracy: 0.8791
Epoch 22/50
100/100 [==============================] - 17s 169ms/step - loss: -0.3451 - binary_crossentropy: 1.3468 - dice_coef: 0.3451 - binary_accuracy: 0.8849 - val_loss: -0.0668 - val_bi

Epoch 34/50
100/100 [==============================] - 17s 166ms/step - loss: -0.4566 - binary_crossentropy: 0.8417 - dice_coef: 0.4566 - binary_accuracy: 0.9370 - val_loss: -0.1106 - val_binary_crossentropy: 1.1133 - val_dice_coef: 0.1106 - val_binary_accuracy: 0.9129
Epoch 35/50
100/100 [==============================] - 17s 166ms/step - loss: -0.4623 - binary_crossentropy: 0.8420 - dice_coef: 0.4623 - binary_accuracy: 0.9374 - val_loss: -0.1647 - val_binary_crossentropy: 1.1584 - val_dice_coef: 0.1647 - val_binary_accuracy: 0.9125
Epoch 36/50
100/100 [==============================] - 17s 167ms/step - loss: -0.4686 - binary_crossentropy: 0.8274 - dice_coef: 0.4686 - binary_accuracy: 0.9387 - val_loss: -0.0995 - val_binary_crossentropy: 1.0159 - val_dice_coef: 0.0995 - val_binary_accuracy: 0.9220
Epoch 37/50
100/100 [==============================] - 17s 166ms/step - loss: -0.4748 - binary_crossentropy: 0.8176 - dice_coef: 0.4748 - binary_accuracy: 0.9394 - val_loss: -0.1331 - val_bi

Epoch 12/50
 98/100 [============================>.] - ETA: 0s - loss: -0.1649 - binary_crossentropy: 0.4325 - dice_coef: 0.1649 - binary_accuracy: 0.9348

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100/100 [==============================] - 17s 167ms/step - loss: -2.5830e-04 - binary_crossentropy: 0.1064 - dice_coef: 2.5830e-04 - binary_accuracy: 0.9909 - val_loss: -0.0034 - val_binary_crossentropy: 0.0072 - val_dice_coef: 0.0034 - val_binary_accuracy: 0.9994
Epoch 31/50
100/100 [==============================] - 17s 172ms/step - loss: -2.3915e-04 - binary_crossentropy: 0.1059 - dice_coef: 2.3915e-04 - binary_accuracy: 0.9910 - val_loss: -0.0034 - val_binary_crossentropy: 0.0072 - val_dice_coef: 0.0034 - val_binary_accuracy: 0.9994
Epoch 32/50
100/100 [==============================] - 17s 169ms/step - loss: -2.6796e-04 - binary_crossentropy: 0.0995 - dice_coef: 2.6796e-04 - binary_accuracy: 0.9916 - val_loss: -0.0034 - val_binary_crossentropy: 0.0072 - val_dice_coef: 0.0034 - val_binary_accuracy: 0.9994
Epoch 33/50
100/100 [==============================] - 17s 167ms/step - loss: -2.6752e-04 - binary_crossentropy: 0.1029 - dice_coef: 2.6752e-04 - binary_accuracy: 0.9914 - val_lo

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/losstest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=dice_coef_loss,\
                      metrics=['binary_crossentropy', dice_coef, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, validation_data=(array([[[..., epochs=50, steps_per_epoch=100, callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 70s 703ms/step - loss: -0.4711 - binary_crossentropy: 1.4252 - dice_coef: 0.4711 - binary_accuracy: 0.4829 - val_loss: -0.4375 - val_binary_crossentropy: 1.5076 - val_dice_coef: 0.4375 - val_binary_accuracy: 0.7255
Epoch 2/50
100/100 [==============================] - 20s 201ms/step - loss: -0.5773 - binary_crossentropy: 1.0468 - dice_coef: 0.5773 - binary_accuracy: 0.7223 - val_loss: -0.4102 - val_binary_crossentropy: 2.0420 - val_dice_coef: 0.4102 - val_binary_accuracy: 0.5918
Epoch 3/50
100/100 [==============================] - 20s 204ms/step - loss: -0.5903 - binary_crossentropy: 1.1907 - dice_coef: 0.5903 - binary_accuracy: 0.7396 - val_loss: -0.4409 - val_binary_crossentropy: 1.4099 - val_dice_coef: 0.4409 - val_binary_accuracy: 0.7087
Epoch 4/50
100/100 [==============================] - 20s 203ms/step - loss: -0.5957 - binary_crossentropy: 1.2825 - dice_coef: 0.5957 - binary_accuracy: 0.7460 - val_loss: -0.4621 - val_binary

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100/100 [==============================] - 20s 197ms/step - loss: -0.7840 - binary_crossentropy: 1.3887 - dice_coef: 0.7840 - binary_accuracy: 0.8783 - val_loss: -0.6021 - val_binary_crossentropy: 1.9302 - val_dice_coef: 0.6021 - val_binary_accuracy: 0.8284
Epoch 41/50
100/100 [==============================] - 19s 194ms/step - loss: -0.7858 - binary_crossentropy: 1.3919 - dice_coef: 0.7858 - binary_accuracy: 0.8792 - val_loss: -0.6158 - val_binary_crossentropy: 1.7634 - val_dice_coef: 0.6158 - val_binary_accuracy: 0.8457
Epoch 42/50
100/100 [==============================] - 19s 193ms/step - loss: -0.7913 - binary_crossentropy: 1.3658 - dice_coef: 0.7913 - binary_accuracy: 0.8826 - val_loss: -0.6022 - val_binary_crossentropy: 1.9048 - val_dice_coef: 0.6022 - val_binary_accuracy: 0.8358
Epoch 43/50
100/100 [==============================] - 20s 196ms/step - loss: -0.7952 - binary_crossentropy: 1.3532 - dice_coef: 0.7952 - binary_accuracy: 0.8840 - val_loss: -0.5932 - val_binary_crossen

...

100/100 [==============================] - 17s 171ms/step - loss: -0.3146 - binary_crossentropy: 0.4230 - dice_coef: 0.3146 - binary_accuracy: 0.9463 - val_loss: -0.2897 - val_binary_crossentropy: 0.4338 - val_dice_coef: 0.2897 - val_binary_accuracy: 0.9486
Epoch 13/50
100/100 [==============================] - 17s 168ms/step - loss: -0.3122 - binary_crossentropy: 0.4295 - dice_coef: 0.3122 - binary_accuracy: 0.9473 - val_loss: -0.2686 - val_binary_crossentropy: 0.5838 - val_dice_coef: 0.2686 - val_binary_accuracy: 0.9355
Epoch 14/50
100/100 [==============================] - 17s 169ms/step - loss: -0.3280 - binary_crossentropy: 0.4468 - dice_coef: 0.3280 - binary_accuracy: 0.9470 - val_loss: -0.2721 - val_binary_crossentropy: 0.5295 - val_dice_coef: 0.2721 - val_binary_accuracy: 0.9398
Epoch 15/50
100/100 [==============================] - 17s 168ms/step - loss: -0.3267 - binary_crossentropy: 0.4413 - dice_coef: 0.3267 - binary_accuracy: 0.9492 - val_loss: -0.2628 - val_binary_crossen

Epoch 20/50
100/100 [==============================] - 17s 168ms/step - loss: -0.4390 - binary_crossentropy: 0.3690 - dice_coef: 0.4390 - binary_accuracy: 0.9673 - val_loss: -0.2151 - val_binary_crossentropy: 0.2500 - val_dice_coef: 0.2151 - val_binary_accuracy: 0.9769
Epoch 21/50
100/100 [==============================] - 17s 169ms/step - loss: -0.4442 - binary_crossentropy: 0.3597 - dice_coef: 0.4442 - binary_accuracy: 0.9685 - val_loss: -0.1627 - val_binary_crossentropy: 0.1833 - val_dice_coef: 0.1627 - val_binary_accuracy: 0.9826
Epoch 22/50
100/100 [==============================] - 17s 172ms/step - loss: -0.4473 - binary_crossentropy: 0.3553 - dice_coef: 0.4473 - binary_accuracy: 0.9690 - val_loss: -0.1556 - val_binary_crossentropy: 0.1565 - val_dice_coef: 0.1556 - val_binary_accuracy: 0.9848
Epoch 23/50
100/100 [==============================] - 17s 168ms/step - loss: -0.4597 - binary_crossentropy: 0.3355 - dice_coef: 0.4597 - binary_accuracy: 0.9709 - val_loss: -0.2557 - val_bi

100/100 [==============================] - 20s 197ms/step - loss: -0.8254 - binary_crossentropy: 1.0011 - dice_coef: 0.8254 - binary_accuracy: 0.8820 - val_loss: -0.7664 - val_binary_crossentropy: 1.4296 - val_dice_coef: 0.7664 - val_binary_accuracy: 0.8363
Epoch 29/50
100/100 [==============================] - 20s 200ms/step - loss: -0.8305 - binary_crossentropy: 0.9814 - dice_coef: 0.8305 - binary_accuracy: 0.8857 - val_loss: -0.7639 - val_binary_crossentropy: 1.4366 - val_dice_coef: 0.7639 - val_binary_accuracy: 0.8345
Epoch 30/50
100/100 [==============================] - 20s 197ms/step - loss: -0.8299 - binary_crossentropy: 0.9947 - dice_coef: 0.8299 - binary_accuracy: 0.8858 - val_loss: -0.7665 - val_binary_crossentropy: 1.4081 - val_dice_coef: 0.7665 - val_binary_accuracy: 0.8414
Epoch 31/50
100/100 [==============================] - 20s 197ms/step - loss: -0.8345 - binary_crossentropy: 0.9726 - dice_coef: 0.8345 - binary_accuracy: 0.8893 - val_loss: -0.7724 - val_binary_crossen

Epoch 6/50
100/100 [==============================] - 17s 168ms/step - loss: -0.7532 - binary_crossentropy: 0.0360 - dice_coef: 0.7532 - binary_accuracy: 0.9937 - val_loss: -0.5844 - val_binary_crossentropy: 0.0888 - val_dice_coef: 0.5844 - val_binary_accuracy: 0.9862
Epoch 7/50
100/100 [==============================] - 17s 168ms/step - loss: -0.7819 - binary_crossentropy: 0.0364 - dice_coef: 0.7819 - binary_accuracy: 0.9939 - val_loss: -0.5734 - val_binary_crossentropy: 0.0934 - val_dice_coef: 0.5734 - val_binary_accuracy: 0.9861
Epoch 8/50
100/100 [==============================] - 17s 171ms/step - loss: -0.7852 - binary_crossentropy: 0.0392 - dice_coef: 0.7852 - binary_accuracy: 0.9939 - val_loss: -0.5882 - val_binary_crossentropy: 0.0946 - val_dice_coef: 0.5882 - val_binary_accuracy: 0.9865
Epoch 9/50
100/100 [==============================] - 17s 167ms/step - loss: -0.7760 - binary_crossentropy: 0.0410 - dice_coef: 0.7760 - binary_accuracy: 0.9939 - val_loss: -0.5834 - val_binary

100/100 [==============================] - 17s 171ms/step - loss: -0.0681 - binary_crossentropy: 0.2457 - dice_coef: 0.0681 - binary_accuracy: 0.9726 - val_loss: -0.1515 - val_binary_crossentropy: 0.2670 - val_dice_coef: 0.1515 - val_binary_accuracy: 0.9683
Epoch 14/50
100/100 [==============================] - 17s 172ms/step - loss: -0.0633 - binary_crossentropy: 0.2881 - dice_coef: 0.0633 - binary_accuracy: 0.9712 - val_loss: -0.1454 - val_binary_crossentropy: 0.3127 - val_dice_coef: 0.1454 - val_binary_accuracy: 0.9664
Epoch 15/50
100/100 [==============================] - 17s 174ms/step - loss: -0.0855 - binary_crossentropy: 0.2828 - dice_coef: 0.0855 - binary_accuracy: 0.9716 - val_loss: -0.1532 - val_binary_crossentropy: 0.2915 - val_dice_coef: 0.1532 - val_binary_accuracy: 0.9691
Epoch 16/50
100/100 [==============================] - 17s 170ms/step - loss: -0.0767 - binary_crossentropy: 0.2836 - dice_coef: 0.0767 - binary_accuracy: 0.9716 - val_loss: -0.1422 - val_binary_crossen

In [15]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/losstest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=dice_coef_loss,\
                      metrics=['binary_crossentropy', dice_coef, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, validation_data=(array([[[..., epochs=50, steps_per_epoch=100, callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 88s 884ms/step - loss: -0.6150 - binary_crossentropy: 0.1439 - dice_coef: 0.6150 - binary_accuracy: 0.9411 - val_loss: -0.7373 - val_binary_crossentropy: 0.0374 - val_dice_coef: 0.7373 - val_binary_accuracy: 0.9910
Epoch 2/50
100/100 [==============================] - 17s 173ms/step - loss: -0.8156 - binary_crossentropy: 0.0270 - dice_coef: 0.8156 - binary_accuracy: 0.9952 - val_loss: -0.7839 - val_binary_crossentropy: 0.0600 - val_dice_coef: 0.7839 - val_binary_accuracy: 0.9911
Epoch 3/50
100/100 [==============================] - 17s 174ms/step - loss: -0.8369 - binary_crossentropy: 0.0285 - dice_coef: 0.8369 - binary_accuracy: 0.9953 - val_loss: -0.7892 - val_binary_crossentropy: 0.0657 - val_dice_coef: 0.7892 - val_binary_accuracy: 0.9912
Epoch 4/50
100/100 [==============================] - 17s 173ms/step - loss: -0.8403 - binary_crossentropy: 0.0356 - dice_coef: 0.8403 - binary_accuracy: 0.9948 - val_loss: -0.7901 - val_binary

Epoch 9/50
100/100 [==============================] - 17s 171ms/step - loss: -0.5510 - binary_crossentropy: 1.9336 - dice_coef: 0.5510 - binary_accuracy: 0.7352 - val_loss: -0.5129 - val_binary_crossentropy: 1.9957 - val_dice_coef: 0.5129 - val_binary_accuracy: 0.7162
Epoch 10/50
100/100 [==============================] - 17s 173ms/step - loss: -0.5522 - binary_crossentropy: 1.9918 - dice_coef: 0.5522 - binary_accuracy: 0.7381 - val_loss: -0.5239 - val_binary_crossentropy: 2.0603 - val_dice_coef: 0.5239 - val_binary_accuracy: 0.7369
Epoch 11/50
100/100 [==============================] - 17s 170ms/step - loss: -0.5558 - binary_crossentropy: 2.0563 - dice_coef: 0.5558 - binary_accuracy: 0.7385 - val_loss: -0.5220 - val_binary_crossentropy: 2.3535 - val_dice_coef: 0.5220 - val_binary_accuracy: 0.7288
Epoch 12/50
100/100 [==============================] - 17s 170ms/step - loss: -0.5581 - binary_crossentropy: 2.0863 - dice_coef: 0.5581 - binary_accuracy: 0.7435 - val_loss: -0.4370 - val_bin

Epoch 17/50
100/100 [==============================] - 20s 196ms/step - loss: -0.5306 - binary_crossentropy: 1.5698 - dice_coef: 0.5306 - binary_accuracy: 0.8210 - val_loss: -0.4955 - val_binary_crossentropy: 1.8361 - val_dice_coef: 0.4955 - val_binary_accuracy: 0.8004
Epoch 18/50
100/100 [==============================] - 20s 197ms/step - loss: -0.5335 - binary_crossentropy: 1.5654 - dice_coef: 0.5335 - binary_accuracy: 0.8234 - val_loss: -0.4801 - val_binary_crossentropy: 1.5729 - val_dice_coef: 0.4801 - val_binary_accuracy: 0.8207
Epoch 19/50
100/100 [==============================] - 20s 196ms/step - loss: -0.5383 - binary_crossentropy: 1.5915 - dice_coef: 0.5383 - binary_accuracy: 0.8260 - val_loss: -0.5016 - val_binary_crossentropy: 1.9930 - val_dice_coef: 0.5016 - val_binary_accuracy: 0.7913
Epoch 20/50
100/100 [==============================] - 20s 198ms/step - loss: -0.5421 - binary_crossentropy: 1.6329 - dice_coef: 0.5421 - binary_accuracy: 0.8261 - val_loss: -0.5125 - val_bi

...

Epoch 19/50
100/100 [==============================] - 20s 201ms/step - loss: -0.1975 - binary_crossentropy: 0.0748 - dice_coef: 0.1975 - binary_accuracy: 0.9928 - val_loss: -0.2801 - val_binary_crossentropy: 0.0417 - val_dice_coef: 0.2801 - val_binary_accuracy: 0.9959
Epoch 20/50
100/100 [==============================] - 20s 199ms/step - loss: -0.1890 - binary_crossentropy: 0.0753 - dice_coef: 0.1890 - binary_accuracy: 0.9929 - val_loss: -0.2871 - val_binary_crossentropy: 0.0440 - val_dice_coef: 0.2871 - val_binary_accuracy: 0.9960
Epoch 21/50
100/100 [==============================] - 20s 198ms/step - loss: -0.1963 - binary_crossentropy: 0.0660 - dice_coef: 0.1963 - binary_accuracy: 0.9941 - val_loss: -0.2107 - val_binary_crossentropy: 0.0826 - val_dice_coef: 0.2107 - val_binary_accuracy: 0.9927
Epoch 22/50
100/100 [==============================] - 21s 206ms/step - loss: -0.2135 - binary_crossentropy: 0.0695 - dice_coef: 0.2135 - binary_accuracy: 0.9939 - val_loss: -0.1875 - val_bi

Epoch 27/50
100/100 [==============================] - 18s 175ms/step - loss: -0.8662 - binary_crossentropy: 0.9893 - dice_coef: 0.8662 - binary_accuracy: 0.8976 - val_loss: -0.8287 - val_binary_crossentropy: 1.1932 - val_dice_coef: 0.8287 - val_binary_accuracy: 0.8680
Epoch 28/50
100/100 [==============================] - 17s 172ms/step - loss: -0.8661 - binary_crossentropy: 0.9911 - dice_coef: 0.8661 - binary_accuracy: 0.8990 - val_loss: -0.8214 - val_binary_crossentropy: 1.3562 - val_dice_coef: 0.8214 - val_binary_accuracy: 0.8611
Epoch 29/50
100/100 [==============================] - 17s 173ms/step - loss: -0.8715 - binary_crossentropy: 0.9799 - dice_coef: 0.8715 - binary_accuracy: 0.9014 - val_loss: -0.8250 - val_binary_crossentropy: 1.4231 - val_dice_coef: 0.8250 - val_binary_accuracy: 0.8604
Epoch 30/50
100/100 [==============================] - 17s 173ms/step - loss: -0.8711 - binary_crossentropy: 0.9827 - dice_coef: 0.8711 - binary_accuracy: 0.9024 - val_loss: -0.8256 - val_bi

Epoch 5/50
100/100 [==============================] - 20s 202ms/step - loss: -0.3598 - binary_crossentropy: 0.7280 - dice_coef: 0.3598 - binary_accuracy: 0.8749 - val_loss: -0.2920 - val_binary_crossentropy: 0.6407 - val_dice_coef: 0.2920 - val_binary_accuracy: 0.8962
Epoch 6/50
100/100 [==============================] - 20s 200ms/step - loss: -0.3706 - binary_crossentropy: 0.7958 - dice_coef: 0.3706 - binary_accuracy: 0.8715 - val_loss: -0.3679 - val_binary_crossentropy: 0.7008 - val_dice_coef: 0.3679 - val_binary_accuracy: 0.8981
Epoch 7/50
100/100 [==============================] - 20s 204ms/step - loss: -0.3666 - binary_crossentropy: 0.8113 - dice_coef: 0.3666 - binary_accuracy: 0.8753 - val_loss: -0.1935 - val_binary_crossentropy: 0.6981 - val_dice_coef: 0.1935 - val_binary_accuracy: 0.9100
Epoch 8/50
100/100 [==============================] - 20s 200ms/step - loss: -0.3791 - binary_crossentropy: 0.8071 - dice_coef: 0.3791 - binary_accuracy: 0.8813 - val_loss: -0.3369 - val_binary

Epoch 13/50
100/100 [==============================] - 21s 206ms/step - loss: -0.2618 - binary_crossentropy: 0.1311 - dice_coef: 0.2618 - binary_accuracy: 0.9880 - val_loss: -0.0407 - val_binary_crossentropy: 5.2573e-05 - val_dice_coef: 0.0407 - val_binary_accuracy: 1.0000
Epoch 14/50
100/100 [==============================] - 20s 203ms/step - loss: -0.2822 - binary_crossentropy: 0.1140 - dice_coef: 0.2822 - binary_accuracy: 0.9893 - val_loss: -0.0862 - val_binary_crossentropy: 2.3665e-05 - val_dice_coef: 0.0862 - val_binary_accuracy: 1.0000
Epoch 15/50
100/100 [==============================] - 20s 204ms/step - loss: -0.2818 - binary_crossentropy: 0.1162 - dice_coef: 0.2818 - binary_accuracy: 0.9893 - val_loss: -0.0381 - val_binary_crossentropy: 5.6334e-05 - val_dice_coef: 0.0381 - val_binary_accuracy: 1.0000
Epoch 16/50
100/100 [==============================] - 20s 203ms/step - loss: -0.2986 - binary_crossentropy: 0.1387 - dice_coef: 0.2986 - binary_accuracy: 0.9879 - val_loss: -0.0

Epoch 20/50
100/100 [==============================] - 20s 202ms/step - loss: -0.3681 - binary_crossentropy: 0.0618 - dice_coef: 0.3681 - binary_accuracy: 0.9949 - val_loss: -0.1472 - val_binary_crossentropy: 0.0518 - val_dice_coef: 0.1472 - val_binary_accuracy: 0.9957
Epoch 21/50
100/100 [==============================] - 20s 201ms/step - loss: -0.4070 - binary_crossentropy: 0.0626 - dice_coef: 0.4070 - binary_accuracy: 0.9948 - val_loss: -0.0905 - val_binary_crossentropy: 0.0422 - val_dice_coef: 0.0905 - val_binary_accuracy: 0.9964
Epoch 22/50
100/100 [==============================] - 20s 200ms/step - loss: -0.4223 - binary_crossentropy: 0.0621 - dice_coef: 0.4223 - binary_accuracy: 0.9951 - val_loss: -0.1169 - val_binary_crossentropy: 0.0380 - val_dice_coef: 0.1169 - val_binary_accuracy: 0.9967
Epoch 23/50
100/100 [==============================] - 20s 200ms/step - loss: -0.4250 - binary_crossentropy: 0.0630 - dice_coef: 0.4250 - binary_accuracy: 0.9952 - val_loss: -0.0813 - val_bi

In [16]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/losstest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=dice_coef_loss,\
                      metrics=['binary_crossentropy', dice_coef, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, validation_data=(array([[[..., epochs=50, steps_per_epoch=100, callbacks=[<keras.ca...)`


Epoch 1/50
100/100 [==============================] - 128s 1s/step - loss: -0.4585 - binary_crossentropy: 0.3329 - dice_coef: 0.4585 - binary_accuracy: 0.8549 - val_loss: -0.2704 - val_binary_crossentropy: 0.3116 - val_dice_coef: 0.2704 - val_binary_accuracy: 0.9207
Epoch 2/50
100/100 [==============================] - 18s 177ms/step - loss: -0.6030 - binary_crossentropy: 0.2426 - dice_coef: 0.6030 - binary_accuracy: 0.9192 - val_loss: -0.4421 - val_binary_crossentropy: 0.2844 - val_dice_coef: 0.4421 - val_binary_accuracy: 0.9255
Epoch 3/50
100/100 [==============================] - 18s 176ms/step - loss: -0.6612 - binary_crossentropy: 0.2644 - dice_coef: 0.6612 - binary_accuracy: 0.9276 - val_loss: -0.5192 - val_binary_crossentropy: 0.4414 - val_dice_coef: 0.5192 - val_binary_accuracy: 0.8943
Epoch 4/50
100/100 [==============================] - 17s 173ms/step - loss: -0.6754 - binary_crossentropy: 0.2744 - dice_coef: 0.6754 - binary_accuracy: 0.9312 - val_loss: -0.3768 - val_binary_c

100/100 [==============================] - 21s 208ms/step - loss: -0.7695 - binary_crossentropy: 0.1150 - dice_coef: 0.7695 - binary_accuracy: 0.9756 - val_loss: -0.6188 - val_binary_crossentropy: 0.3362 - val_dice_coef: 0.6188 - val_binary_accuracy: 0.9529
Epoch 10/50
100/100 [==============================] - 20s 204ms/step - loss: -0.7789 - binary_crossentropy: 0.1167 - dice_coef: 0.7789 - binary_accuracy: 0.9761 - val_loss: -0.7723 - val_binary_crossentropy: 0.1484 - val_dice_coef: 0.7723 - val_binary_accuracy: 0.9720
Epoch 11/50
100/100 [==============================] - 21s 205ms/step - loss: -0.7844 - binary_crossentropy: 0.1134 - dice_coef: 0.7844 - binary_accuracy: 0.9773 - val_loss: -0.8017 - val_binary_crossentropy: 0.1276 - val_dice_coef: 0.8017 - val_binary_accuracy: 0.9741
Epoch 12/50
100/100 [==============================] - 20s 204ms/step - loss: -0.7876 - binary_crossentropy: 0.1173 - dice_coef: 0.7876 - binary_accuracy: 0.9776 - val_loss: -0.8158 - val_binary_crossen

Epoch 17/50
100/100 [==============================] - 21s 208ms/step - loss: -0.8478 - binary_crossentropy: 0.0275 - dice_coef: 0.8478 - binary_accuracy: 0.9948 - val_loss: -0.8671 - val_binary_crossentropy: 0.0097 - val_dice_coef: 0.8671 - val_binary_accuracy: 0.9976
Epoch 18/50
100/100 [==============================] - 21s 205ms/step - loss: -0.8620 - binary_crossentropy: 0.0239 - dice_coef: 0.8620 - binary_accuracy: 0.9954 - val_loss: -0.8915 - val_binary_crossentropy: 0.0071 - val_dice_coef: 0.8915 - val_binary_accuracy: 0.9981
Epoch 19/50
100/100 [==============================] - 21s 206ms/step - loss: -0.8523 - binary_crossentropy: 0.0302 - dice_coef: 0.8523 - binary_accuracy: 0.9948 - val_loss: -0.8895 - val_binary_crossentropy: 0.0078 - val_dice_coef: 0.8895 - val_binary_accuracy: 0.9979
Epoch 20/50
100/100 [==============================] - 21s 206ms/step - loss: -0.8662 - binary_crossentropy: 0.0262 - dice_coef: 0.8662 - binary_accuracy: 0.9954 - val_loss: -0.8562 - val_bi

...

100/100 [==============================] - 28s 278ms/step - loss: -0.7017 - binary_crossentropy: 0.0704 - dice_coef: 0.7017 - binary_accuracy: 0.9951 - val_loss: -0.7023 - val_binary_crossentropy: 0.0788 - val_dice_coef: 0.7023 - val_binary_accuracy: 0.9948
Epoch 44/50
100/100 [==============================] - 23s 229ms/step - loss: -0.7166 - binary_crossentropy: 0.0647 - dice_coef: 0.7166 - binary_accuracy: 0.9954 - val_loss: -0.7015 - val_binary_crossentropy: 0.0790 - val_dice_coef: 0.7015 - val_binary_accuracy: 0.9948
Epoch 45/50
100/100 [==============================] - 48s 482ms/step - loss: -0.7070 - binary_crossentropy: 0.0684 - dice_coef: 0.7070 - binary_accuracy: 0.9952 - val_loss: -0.7033 - val_binary_crossentropy: 0.0789 - val_dice_coef: 0.7033 - val_binary_accuracy: 0.9948
Epoch 46/50
100/100 [==============================] - 64s 636ms/step - loss: -0.7057 - binary_crossentropy: 0.0683 - dice_coef: 0.7057 - binary_accuracy: 0.9952 - val_loss: -0.6995 - val_binary_crossen

Epoch 21/50
100/100 [==============================] - 38s 379ms/step - loss: -0.8154 - binary_crossentropy: 0.0770 - dice_coef: 0.8154 - binary_accuracy: 0.9880 - val_loss: -0.8297 - val_binary_crossentropy: 0.0573 - val_dice_coef: 0.8297 - val_binary_accuracy: 0.9898
Epoch 22/50
100/100 [==============================] - 23s 226ms/step - loss: -0.8179 - binary_crossentropy: 0.0788 - dice_coef: 0.8179 - binary_accuracy: 0.9879 - val_loss: -0.7420 - val_binary_crossentropy: 0.1043 - val_dice_coef: 0.7420 - val_binary_accuracy: 0.9856
Epoch 23/50
100/100 [==============================] - 81s 811ms/step - loss: -0.8225 - binary_crossentropy: 0.0763 - dice_coef: 0.8225 - binary_accuracy: 0.9883 - val_loss: -0.6835 - val_binary_crossentropy: 0.1520 - val_dice_coef: 0.6835 - val_binary_accuracy: 0.9820
Epoch 24/50
100/100 [==============================] - 23s 230ms/step - loss: -0.8254 - binary_crossentropy: 0.0770 - dice_coef: 0.8254 - binary_accuracy: 0.9884 - val_loss: -0.8326 - val_bi



label: 12
{12: 1.0}
composition with label != 0
Y_train 0.9001610305958132 0.031516203703703706
composition with label != 0
Y_val 0.9428571428571428 0.03363392857142857
composition with label != 0
Y_test 0.8959537572254336 0.03351427023121387
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)
Epoch 1/50
100/100 [==============================] - 954s 10s/step - loss: -0.2690 - binary_crossentropy: 0.2180 - dice_coef: 0.2690 - binary_accuracy: 0.9068 - val_loss: -0.1560 - val_binary_crossentropy: 0.1474 - val_dice_coef: 0.1560 - val_binary_accuracy: 0.9587
Epoch 2/50
100/100 [==============================] - 21s 211ms/step - loss: -0.5112 - binary_crossentropy: 0.0974 - dice_coef: 0.5112 - binary_accuracy: 0.9674 - val_loss: -0.3277 - val_binary_crossentropy: 0.1743 - val_dice_coef: 0.3277 - val_binary_accuracy: 0.9533
Epoch 3/50
100/100 [============

Epoch 30/50
100/100 [==============================] - 20s 199ms/step - loss: -0.8665 - binary_crossentropy: 0.0422 - dice_coef: 0.8665 - binary_accuracy: 0.9921 - val_loss: -0.8784 - val_binary_crossentropy: 0.0390 - val_dice_coef: 0.8784 - val_binary_accuracy: 0.9924
Epoch 31/50
100/100 [==============================] - 36s 357ms/step - loss: -0.8709 - binary_crossentropy: 0.0421 - dice_coef: 0.8709 - binary_accuracy: 0.9921 - val_loss: -0.8724 - val_binary_crossentropy: 0.0414 - val_dice_coef: 0.8724 - val_binary_accuracy: 0.9923
Epoch 32/50
100/100 [==============================] - 72s 720ms/step - loss: -0.8705 - binary_crossentropy: 0.0429 - dice_coef: 0.8705 - binary_accuracy: 0.9921 - val_loss: -0.8497 - val_binary_crossentropy: 0.0597 - val_dice_coef: 0.8497 - val_binary_accuracy: 0.9898
Epoch 33/50
100/100 [==============================] - 23s 233ms/step - loss: -0.8691 - binary_crossentropy: 0.0452 - dice_coef: 0.8691 - binary_accuracy: 0.9918 - val_loss: -0.8851 - val_bi

KeyboardInterrupt: 

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 13
{13: 1.0}
composition with label != 0
Y_train 0.8470209339774557 0.03495420692431562
composition with label != 0
Y_val 0.8 0.031046875
composition with label != 0
Y_test 0.8786127167630058 0.031108200867052023
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., validation_data=(array([[[..., epochs=50, verbose=1, callbacks=[<keras.ca..., steps_per_epoch=100)`


Epoch 1/50
100/100 [==============================] - 33s 326ms/step - loss: -0.3396 - binary_crossentropy: 0.3348 - dice_coef: 0.3396 - binary_accuracy: 0.8614 - val_loss: -0.3898 - val_binary_crossentropy: 0.1108 - val_dice_coef: 0.3898 - val_binary_accuracy: 0.9724
Epoch 2/50
100/100 [==============================] - 22s 221ms/step - loss: -0.6263 - binary_crossentropy: 0.0932 - dice_coef: 0.6263 - binary_accuracy: 0.9752 - val_loss: -0.5336 - val_binary_crossentropy: 0.0968 - val_dice_coef: 0.5336 - val_binary_accuracy: 0.9780
Epoch 3/50
100/100 [==============================] - 21s 215ms/step - loss: -0.7130 - binary_crossentropy: 0.0788 - dice_coef: 0.7130 - binary_accuracy: 0.9807 - val_loss: -0.6242 - val_binary_crossentropy: 0.1754 - val_dice_coef: 0.6242 - val_binary_accuracy: 0.9669
Epoch 4/50
100/100 [==============================] - 21s 210ms/step - loss: -0.7549 - binary_crossentropy: 0.0723 - dice_coef: 0.7549 - binary_accuracy: 0.9834 - val_loss: -0.7759 - val_binary

100/100 [==============================] - 19s 190ms/step - loss: -0.9152 - binary_crossentropy: 0.0340 - dice_coef: 0.9152 - binary_accuracy: 0.9942 - val_loss: -0.8965 - val_binary_crossentropy: 0.0428 - val_dice_coef: 0.8965 - val_binary_accuracy: 0.9937
Epoch 32/50
100/100 [==============================] - 19s 192ms/step - loss: -0.9159 - binary_crossentropy: 0.0335 - dice_coef: 0.9159 - binary_accuracy: 0.9943 - val_loss: -0.9148 - val_binary_crossentropy: 0.0287 - val_dice_coef: 0.9148 - val_binary_accuracy: 0.9947
Epoch 33/50
100/100 [==============================] - 19s 190ms/step - loss: -0.9172 - binary_crossentropy: 0.0342 - dice_coef: 0.9172 - binary_accuracy: 0.9943 - val_loss: -0.8801 - val_binary_crossentropy: 0.0488 - val_dice_coef: 0.8801 - val_binary_accuracy: 0.9930
Epoch 34/50
100/100 [==============================] - 19s 192ms/step - loss: -0.9210 - binary_crossentropy: 0.0311 - dice_coef: 0.9210 - binary_accuracy: 0.9947 - val_loss: -0.8639 - val_binary_crossen

Epoch 9/50
100/100 [==============================] - 16s 163ms/step - loss: -0.7304 - binary_crossentropy: 0.1497 - dice_coef: 0.7304 - binary_accuracy: 0.9721 - val_loss: -0.7665 - val_binary_crossentropy: 0.1109 - val_dice_coef: 0.7665 - val_binary_accuracy: 0.9748
Epoch 10/50
100/100 [==============================] - 16s 164ms/step - loss: -0.7380 - binary_crossentropy: 0.1478 - dice_coef: 0.7380 - binary_accuracy: 0.9734 - val_loss: -0.8008 - val_binary_crossentropy: 0.0839 - val_dice_coef: 0.8008 - val_binary_accuracy: 0.9810
Epoch 11/50
100/100 [==============================] - 16s 161ms/step - loss: -0.7470 - binary_crossentropy: 0.1453 - dice_coef: 0.7470 - binary_accuracy: 0.9744 - val_loss: -0.6468 - val_binary_crossentropy: 0.1905 - val_dice_coef: 0.6468 - val_binary_accuracy: 0.9687
Epoch 12/50
100/100 [==============================] - 16s 162ms/step - loss: -0.7626 - binary_crossentropy: 0.1349 - dice_coef: 0.7626 - binary_accuracy: 0.9763 - val_loss: -0.7893 - val_bin

...

Epoch 5/50
100/100 [==============================] - 16s 160ms/step - loss: -0.6785 - binary_crossentropy: 0.2280 - dice_coef: 0.6785 - binary_accuracy: 0.9451 - val_loss: -0.6484 - val_binary_crossentropy: 0.1723 - val_dice_coef: 0.6484 - val_binary_accuracy: 0.9538
Epoch 6/50
100/100 [==============================] - 16s 162ms/step - loss: -0.6942 - binary_crossentropy: 0.2196 - dice_coef: 0.6942 - binary_accuracy: 0.9494 - val_loss: -0.6330 - val_binary_crossentropy: 0.2287 - val_dice_coef: 0.6330 - val_binary_accuracy: 0.9434
Epoch 7/50
100/100 [==============================] - 16s 160ms/step - loss: -0.7148 - binary_crossentropy: 0.2182 - dice_coef: 0.7148 - binary_accuracy: 0.9514 - val_loss: -0.4548 - val_binary_crossentropy: 0.4597 - val_dice_coef: 0.4548 - val_binary_accuracy: 0.9175
Epoch 8/50
100/100 [==============================] - 16s 159ms/step - loss: -0.7274 - binary_crossentropy: 0.2212 - dice_coef: 0.7274 - binary_accuracy: 0.9531 - val_loss: -0.7586 - val_binary

100/100 [==============================] - 16s 158ms/step - loss: -0.8642 - binary_crossentropy: 0.1541 - dice_coef: 0.8642 - binary_accuracy: 0.9770 - val_loss: -0.8700 - val_binary_crossentropy: 0.1175 - val_dice_coef: 0.8700 - val_binary_accuracy: 0.9798
Epoch 36/50
100/100 [==============================] - 16s 160ms/step - loss: -0.8779 - binary_crossentropy: 0.1300 - dice_coef: 0.8779 - binary_accuracy: 0.9795 - val_loss: -0.8111 - val_binary_crossentropy: 0.2271 - val_dice_coef: 0.8111 - val_binary_accuracy: 0.9694
Epoch 37/50
100/100 [==============================] - 16s 159ms/step - loss: -0.8683 - binary_crossentropy: 0.1514 - dice_coef: 0.8683 - binary_accuracy: 0.9779 - val_loss: -0.7946 - val_binary_crossentropy: 0.2484 - val_dice_coef: 0.7946 - val_binary_accuracy: 0.9683
Epoch 38/50
100/100 [==============================] - 16s 158ms/step - loss: -0.8649 - binary_crossentropy: 0.1608 - dice_coef: 0.8649 - binary_accuracy: 0.9772 - val_loss: -0.9025 - val_binary_crossen

Epoch 13/50
100/100 [==============================] - 16s 160ms/step - loss: -0.7994 - binary_crossentropy: 0.1013 - dice_coef: 0.7994 - binary_accuracy: 0.9822 - val_loss: -0.7627 - val_binary_crossentropy: 0.0635 - val_dice_coef: 0.7627 - val_binary_accuracy: 0.9881
Epoch 14/50
100/100 [==============================] - 16s 161ms/step - loss: -0.8089 - binary_crossentropy: 0.0985 - dice_coef: 0.8089 - binary_accuracy: 0.9825 - val_loss: -0.7775 - val_binary_crossentropy: 0.0704 - val_dice_coef: 0.7775 - val_binary_accuracy: 0.9863
Epoch 15/50
100/100 [==============================] - 16s 161ms/step - loss: -0.8139 - binary_crossentropy: 0.0973 - dice_coef: 0.8139 - binary_accuracy: 0.9831 - val_loss: -0.7256 - val_binary_crossentropy: 0.1291 - val_dice_coef: 0.7256 - val_binary_accuracy: 0.9802
Epoch 16/50
100/100 [==============================] - 16s 161ms/step - loss: -0.8192 - binary_crossentropy: 0.0929 - dice_coef: 0.8192 - binary_accuracy: 0.9838 - val_loss: -0.7455 - val_bi

100/100 [==============================] - 16s 157ms/step - loss: -0.9071 - binary_crossentropy: 0.0452 - dice_coef: 0.9071 - binary_accuracy: 0.9919 - val_loss: -0.8221 - val_binary_crossentropy: 0.0831 - val_dice_coef: 0.8221 - val_binary_accuracy: 0.9889
Epoch 44/50
100/100 [==============================] - 16s 158ms/step - loss: -0.9024 - binary_crossentropy: 0.0502 - dice_coef: 0.9024 - binary_accuracy: 0.9914 - val_loss: -0.8965 - val_binary_crossentropy: 0.0300 - val_dice_coef: 0.8965 - val_binary_accuracy: 0.9940
Epoch 45/50
100/100 [==============================] - 16s 156ms/step - loss: -0.9113 - binary_crossentropy: 0.0427 - dice_coef: 0.9113 - binary_accuracy: 0.9923 - val_loss: -0.8793 - val_binary_crossentropy: 0.0421 - val_dice_coef: 0.8793 - val_binary_accuracy: 0.9925
Epoch 46/50
100/100 [==============================] - 16s 156ms/step - loss: -0.9141 - binary_crossentropy: 0.0422 - dice_coef: 0.9141 - binary_accuracy: 0.9924 - val_loss: -0.8486 - val_binary_crossen

100/100 [==============================] - 19s 190ms/step - loss: -0.8423 - binary_crossentropy: 0.2293 - dice_coef: 0.8423 - binary_accuracy: 0.9637 - val_loss: -0.8171 - val_binary_crossentropy: 0.2866 - val_dice_coef: 0.8171 - val_binary_accuracy: 0.9536
Epoch 22/50
100/100 [==============================] - 19s 190ms/step - loss: -0.8535 - binary_crossentropy: 0.2140 - dice_coef: 0.8535 - binary_accuracy: 0.9655 - val_loss: -0.7644 - val_binary_crossentropy: 0.4311 - val_dice_coef: 0.7644 - val_binary_accuracy: 0.9409
Epoch 23/50
100/100 [==============================] - 19s 191ms/step - loss: -0.8483 - binary_crossentropy: 0.2242 - dice_coef: 0.8483 - binary_accuracy: 0.9649 - val_loss: -0.7835 - val_binary_crossentropy: 0.4142 - val_dice_coef: 0.7835 - val_binary_accuracy: 0.9434
Epoch 24/50
100/100 [==============================] - 19s 190ms/step - loss: -0.8583 - binary_crossentropy: 0.2091 - dice_coef: 0.8583 - binary_accuracy: 0.9673 - val_loss: -0.5812 - val_binary_crossen